In [4]:
import pandas as pd
import numpy as np
import shap

from colorama import Style, Fore
blk = Style.BRIGHT + Fore.BLACK
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [5]:
greeks_df = pd.read_csv('/home/howt51/work/vscode_micromamba/kaggle/data/icr-identify-age-related-conditions/greeks.csv')
train_df = pd.read_csv('/home/howt51/work/vscode_micromamba/kaggle/data/icr-identify-age-related-conditions/train.csv')
test_df = pd.read_csv('/home/howt51/work/vscode_micromamba/kaggle/data/icr-identify-age-related-conditions/test.csv')

In [6]:
tmp = train_df.copy().drop('Id', axis=1)
y = tmp['Class']
X = tmp.drop(['Class'], axis=1)

In [7]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

def one_hot_encode(df, columns=None):
    # If no specific columns specified, find columns with object datatype
    if columns == None:
        columns = df.select_dtypes(include=[object]).columns.tolist()
        print('found object olumns:')
        for i in columns:
            print(i)

    # Initialize OneHotEncoder from sklearn
    ohe = OneHotEncoder(sparse=False)

    for column in columns:
        # Fit and transform the OneHotEncoder
        ohe_df = pd.DataFrame(ohe.fit_transform(df[[column]]))

        # Get feature names and create column names
        ohe_df.columns = ohe.get_feature_names_out() #get_feature_names([column])

        # Concatenate the one-hot encoded columns to original dataframe
        df = pd.concat([df, ohe_df], axis=1)

        # Drop original column from dataframe
        df = df.drop([column], axis=1)

    return df

X = one_hot_encode(X)

found object olumns:
EJ


`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [8]:
X = X.drop(['AH', 'AY', 'AZ', 'BD ', 'BZ', 'DV', 'GE', 'EJ_A', 'EJ_B'], axis=1)
y = y

In [3]:
def competition_log_loss(y_true, y_pred):
    # y_true: correct labels 0, 1
    # y_pred: predicted probabilities of class=1
    # Implements the Evaluation equation with w_0 = w_1 = 1.
    # Calculate the number of observations for each class
    N_0 = np.sum(1 - y_true)
    N_1 = np.sum(y_true)
    # Calculate the predicted probabilities for each class
    p_1 = np.clip(y_pred, 1e-15, 1 - 1e-15)
    p_0 = 1 - p_1
    # Calculate the average log loss for each class
    log_loss_0 = -np.sum((1 - y_true) * np.log(p_0)) / N_0
    log_loss_1 = -np.sum(y_true * np.log(p_1)) / N_1
    # return the (not further weighted) average of the averages
    return (log_loss_0 + log_loss_1)/2

def custom_metric(
        X_val, y_val, estimator, labels,
        X_train, y_train, weight_val=None, weight_train=None,
        *args,
    ):
        from sklearn.metrics import log_loss
        import time
    
        start = time.time()
        y_pred = estimator.predict_proba(X_val)[:,1]
        pred_time = (time.time() - start) / len(X_val)
        val_loss = competition_log_loss(y_val, y_pred)
        #log_loss(y_val, y_pred, labels=labels, sample_weight=weight_val)
        y_pred = estimator.predict_proba(X_train)[:,1]
        train_loss = competition_log_loss(y_train, y_pred)
        alpha = 0.5
        return val_loss * (1 + alpha) - alpha * train_loss, {
            "val_loss": val_loss,
            "train_loss": train_loss,
            "pred_time": pred_time,
        }

In [8]:
from flaml import AutoML
automl = AutoML()
automl_settings = {
    "task": "classification",
    "starting_points": "data",
    "estimator_list": ["lgbm"],
    "time_budget": 1800,
    "max_iter": 150,
    "metric": custom_metric,
}
automl.fit(X, y, **automl_settings)
print("----")
print(automl.best_estimator)
print(automl.best_config)

lgbm_automl = automl

[flaml.automl.logger: 07-19 22:22:43] {1693} INFO - task = classification
[flaml.automl.logger: 07-19 22:22:43] {1700} INFO - Data split method: stratified
[flaml.automl.logger: 07-19 22:22:43] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 07-19 22:22:43] {1801} INFO - Minimizing error metric: customized metric
[flaml.automl.logger: 07-19 22:22:43] {129} WARNING - Starting points outside of the search space are removed. Remaining starting points for <class 'flaml.automl.model.LGBMEstimator'>: []
[flaml.automl.logger: 07-19 22:22:43] {1911} INFO - List of ML learners in AutoML Run: ['lgbm']
[flaml.automl.logger: 07-19 22:22:43] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-19 22:22:44] {2347} INFO - Estimated sufficient time budget=3388s. Estimated necessary time budget=3s.
[flaml.automl.logger: 07-19 22:22:44] {2394} INFO -  at 0.4s,	estimator lgbm's best error=0.7880,	best estimator lgbm's best error=0.7880
[flaml.automl.logger: 07-19 22:22:44] {

In [ ]:
lgbm
{'n_estimators': 80, 'num_leaves': 4, 'min_child_samples': 11, 'learning_rate': 0.15698093294147195, 'log_max_bin': 7, 'colsample_bytree': 0.7341405370826501, 'reg_alpha': 0.0014116842931715303, 'reg_lambda': 6.377091034346681}

In [9]:
from flaml import AutoML
automl = AutoML()
automl_settings = {
    "task": "classification",
    "starting_points": "data",
    "estimator_list": ["extra_tree"],
    "time_budget": 1800,
    "max_iter": 150,
    "metric": custom_metric,
}
automl.fit(X, y, **automl_settings)
print("----")
print(automl.best_estimator)
print(automl.best_config)

et_automl = automl

[flaml.automl.logger: 07-19 22:25:35] {1693} INFO - task = classification
[flaml.automl.logger: 07-19 22:25:35] {1700} INFO - Data split method: stratified
[flaml.automl.logger: 07-19 22:25:35] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 07-19 22:25:35] {1801} INFO - Minimizing error metric: customized metric
[flaml.automl.logger: 07-19 22:25:35] {129} WARNING - Starting points outside of the search space are removed. Remaining starting points for <class 'flaml.automl.model.ExtraTreesEstimator'>: []
[flaml.automl.logger: 07-19 22:25:35] {1911} INFO - List of ML learners in AutoML Run: ['extra_tree']
[flaml.automl.logger: 07-19 22:25:35] {2221} INFO - iteration 0, current learner extra_tree
[flaml.automl.logger: 07-19 22:25:35] {2347} INFO - Estimated sufficient time budget=3380s. Estimated necessary time budget=3s.
[flaml.automl.logger: 07-19 22:25:35] {2394} INFO -  at 0.4s,	estimator extra_tree's best error=0.8174,	best estimator extra_tree's best error=0.8174
[flaml.au

In [ ]:
extra_tree
{'n_estimators': 4, 'max_features': 0.1, 'max_leaves': 19, 'criterion': 'entropy'}

In [10]:
from flaml import AutoML
automl = AutoML()
automl_settings = {
    "task": "classification",
    "starting_points": "data",
    "estimator_list": ["rf"],
    "time_budget": 1800,
    "max_iter": 150,
    "metric": custom_metric,
}
automl.fit(X, y, **automl_settings)
print("----")
print(automl.best_estimator)
print(automl.best_config)

lgbm_automl = automl

[flaml.automl.logger: 07-19 22:26:19] {1693} INFO - task = classification
[flaml.automl.logger: 07-19 22:26:19] {1700} INFO - Data split method: stratified
[flaml.automl.logger: 07-19 22:26:19] {1703} INFO - Evaluation method: cv
[flaml.automl.logger: 07-19 22:26:19] {1801} INFO - Minimizing error metric: customized metric
[flaml.automl.logger: 07-19 22:26:19] {1911} INFO - List of ML learners in AutoML Run: ['rf']
[flaml.automl.logger: 07-19 22:26:19] {2221} INFO - iteration 0, current learner rf
[flaml.automl.logger: 07-19 22:26:21] {2347} INFO - Estimated sufficient time budget=18116s. Estimated necessary time budget=18s.
[flaml.automl.logger: 07-19 22:26:21] {2394} INFO -  at 1.9s,	estimator rf's best error=0.6224,	best estimator rf's best error=0.6224
[flaml.automl.logger: 07-19 22:26:21] {2221} INFO - iteration 1, current learner rf
[flaml.automl.logger: 07-19 22:26:21] {2394} INFO -  at 2.2s,	estimator rf's best error=0.6224,	best estimator rf's best error=0.6224
[flaml.automl.l

In [ ]:
rf
{'n_estimators': 48, 'max_leaves': 10, 'max_features': 0.38591274966287237, 'criterion': 'entropy'}

In [9]:
import math
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA



def generate_pca_columns(dataframe:pd.DataFrame, columns:list, threshold:float):
    if columns is None:
        columns = dataframe.columns
    # Select the specified columns from the DataFrame
    selected_data = dataframe[columns]

    # Apply PCA to the selected data
    pca = PCA()
    pca.fit(selected_data)

    # Calculate the explained variance ratio for each PCA dimension
    explained_variance_ratio = pca.explained_variance_ratio_

    # Determine the number of significant PCA dimensions based on the threshold
    num_significant_dimensions = sum(explained_variance_ratio >= threshold)

    # Generate the additional columns based on the significant PCA dimensions
    pca_features = pca.transform(selected_data)[:, :num_significant_dimensions]
    pca_columns = [f'PCA_{i+1}' for i in range(num_significant_dimensions)]
    pca_dataframe = pd.DataFrame(pca_features, columns=pca_columns)

    # Concatenate the original DataFrame with the PCA DataFrame
    result_dataframe = pd.concat([dataframe, pca_dataframe], axis=1)

In [28]:
import pandas as pd

def enhance_describe(df:pd.DataFrame, column:str):
    if columns is None:
        columns = df.columns
    df = df[columns]
    df1 = df.describe(include = 'all')
    df1.loc['dtype'] = df.dtypes
    df1.loc['size'] = len(df)
    df1.loc['% count'] = df.isnull().mean()
    return df1

In [29]:
enhance_describe(X)

TypeError: enhance_describe() missing 1 required positional argument: 'column'

In [13]:
generate_pca_columns(X, None, 0.95)

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values